In [4]:
!pip3 install --upgrade certifi

  Obtaining dependency information for certifi from https://files.pythonhosted.org/packages/1c/d5/c84e1a17bf61d4df64ca866a1c9a913874b4e9bdc131ec689a0ad013fb36/certifi-2024.7.4-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 1.0 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: certifi
    Found existing installation: certifi 2024.2.2
    Uninstalling certifi-2024.2.2:
      Successfully uninstalled certifi-2024.2.2

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [23]:
import pandas as pd
import numpy as np

In [7]:
import pandas as pd
import requests
from io import StringIO

# URL of the CSV file
url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv?raw=1'

# Fetch the CSV file using requests
response = requests.get(url)
response.raise_for_status()  # Check if the request was successful

# Use StringIO to read the CSV content
csv_data = StringIO(response.text)

# Load the CSV data into a pandas DataFrame
df = pd.read_csv(csv_data)

# Display the DataFrame
print(df)

                                             answer_llm  \
0     You can sign up for the course by visiting the...   
1     You can sign up using the link provided in the...   
2     Yes, there is an FAQ for the Machine Learning ...   
3     The context does not provide any specific info...   
4     To structure your questions and answers for th...   
...                                                 ...   
1825  Some suggested titles for listing the Machine ...   
1826  It is best advised that you do not list the Ma...   
1827  You can incorporate your Machine Learning Zoom...   
1828  The advice on including a project link in a CV...   
1829  The suggestion to showcase progress through Li...   

                                            answer_orig  document  \
0     Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
1     Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
2     Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
3     Machine L

In [8]:
df = df.iloc[:300]

In [9]:
len(df)

300

In [10]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [11]:
answer_llm = df.iloc[0].answer_llm

In [16]:
v = embedding_model.encode(answer_llm)

In [17]:
v

array([-4.22446579e-01, -2.24855691e-01, -3.24058741e-01, -2.84758866e-01,
        7.25708902e-03,  1.01186641e-01,  1.03716828e-01, -1.89983517e-01,
       -2.80596483e-02,  2.71588504e-01, -1.15337148e-01,  1.14666402e-01,
       -8.49587694e-02,  3.32365334e-01,  5.52724376e-02, -2.22195536e-01,
       -1.42540991e-01,  1.02519184e-01, -1.52333423e-01, -2.02912375e-01,
        1.98422819e-02,  8.38149786e-02, -5.68631887e-01,  2.32843235e-02,
       -1.67292640e-01, -2.39256412e-01, -8.05465057e-02,  2.57083327e-02,
       -8.15466121e-02, -7.39289969e-02, -2.61550128e-01,  1.92576181e-02,
        3.22909296e-01,  1.90357268e-01, -9.34726486e-05, -2.13165566e-01,
        2.88943667e-02, -1.79528147e-02, -5.92762418e-02,  1.99918032e-01,
       -4.75170687e-02,  1.71633765e-01, -2.45915428e-02, -9.38056633e-02,
       -3.57002944e-01,  1.33263484e-01,  1.94045842e-01, -1.18530914e-01,
        4.56915259e-01,  1.47728145e-01,  3.35945159e-01, -1.86959863e-01,
        2.45954141e-01, -

In [18]:
evaluations = []

In [19]:
df.head(1)

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp


In [20]:
for index, row in df.iterrows():
    answer_llm = row['answer_llm']
    answer_orig = row['answer_orig']

    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_llm)

    dt = v_llm.dot(v_orig)
    evaluations.append(dt)

In [22]:
len(evaluations)

300

In [ ]:
evaluations

In [24]:
np.percentile(evaluations, 75)

40.95609760284424

Q3 - Consine Similarity, Normalized vector

In [26]:
def norm_vector(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm

    return v_norm

In [27]:
evaluations_norm = []

for index, row in df.iterrows():
    answer_llm = row['answer_llm']
    answer_orig = row['answer_orig']

    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_llm)

    v_llm_norm = norm_vector(v_llm)
    v_orig_norm = norm_vector(v_orig)

    dt = v_llm_norm.dot(v_orig_norm)
    evaluations_norm.append(dt)

In [28]:
evaluations_norm

[0.99999994,
 1.0000001,
 0.9999998,
 1.0,
 1.0000001,
 0.99999994,
 0.99999976,
 1.0000001,
 1.0,
 0.9999999,
 1.0,
 0.9999999,
 0.99999994,
 0.9999999,
 1.0,
 1.0,
 1.0000001,
 1.0,
 1.0,
 1.0000001,
 1.0,
 1.0,
 1.0,
 1.0,
 0.99999976,
 1.0,
 1.0,
 1.0,
 1.0000001,
 1.0000001,
 1.0000001,
 0.9999999,
 0.9999999,
 1.0,
 0.9999999,
 0.99999994,
 0.99999994,
 1.0,
 1.0000001,
 0.9999999,
 0.9999998,
 1.0000001,
 1.0,
 1.0000002,
 1.0,
 1.0,
 1.0000001,
 1.0,
 1.0,
 0.9999999,
 1.0000001,
 1.0,
 1.0,
 1.0,
 1.0000001,
 1.0,
 1.0,
 1.0000001,
 1.0,
 1.0000001,
 1.0,
 0.99999994,
 1.0,
 0.99999994,
 1.0,
 1.0,
 1.0,
 0.99999994,
 0.9999998,
 1.0000001,
 0.9999999,
 1.0,
 1.0,
 1.0,
 0.9999999,
 0.9999999,
 1.0,
 0.99999994,
 1.0,
 1.0,
 1.0,
 1.0000001,
 1.0,
 1.0000001,
 0.99999994,
 1.0,
 1.0,
 0.99999994,
 0.99999994,
 1.0,
 0.9999999,
 1.0,
 1.0,
 0.99999994,
 1.0,
 1.0000001,
 0.99999994,
 0.99999994,
 0.99999994,
 0.9999999,
 1.0,
 0.9999999,
 1.0,
 1.0,
 0.99999994,
 1.0,
 1.0,
 1.

In [32]:
np.percentile(evaluations_norm, 75)

1.0

Q4. Rouge

Now we will explore an alternative metric - the ROUGE score.

This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

It can give a more nuanced view of text similarity than just cosine similarity alone.

We don't need to implement it ourselves, there's a python package for it:

pip install rouge
(The latest version at the moment of writing is 1.0.1)

Let's compute the ROUGE score between the answers at the index 10 of our dataframe (doc_id=5170565b)

from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
There are three scores: rouge-1, rouge-2 and rouge-l, and precision, recall and F1 score for each.

rouge-1 - the overlap of unigrams,
rouge-2 - bigrams,
rouge-l - the longest common subsequence
What's the F score for rouge-1?

0.35
0.45
0.55
0.65


In [33]:
!pip3 install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for rouge from https://files.pythonhosted.org/packages/32/7c/650ae86f92460e9e8ef969cc5008b24798dcf56a9a8947d04c78f550b3f5/rouge-1.0.1-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [37]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(df.iloc[10]['answer_llm'], df.iloc[10]['answer_orig'])[0]

In [38]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [49]:
f1 = []
for k, v in scores.items():
    for k_1, v in v.items():
        if k_1 == 'f':
            f1.append(v)
print(f1)

print(np.mean(f1))

[0.45454544954545456, 0.21621621121621637, 0.393939388939394]
0.35490034990035496


Q6. Average rouge score for all the data points

Now let's compute the score for all the records and create a dataframe from them.

What's the average rouge_2 across all the records?

In [50]:
rouge_scores = []

for index, row in df.iterrows():
    score = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    rouge_scores.append(score)

In [53]:
rouge_scores

[{'rouge-1': {'r': 0.061224489795918366,
   'p': 0.21428571428571427,
   'f': 0.09523809178130524},
  'rouge-2': {'r': 0.017543859649122806,
   'p': 0.07142857142857142,
   'f': 0.028169010918468917},
  'rouge-l': {'r': 0.061224489795918366,
   'p': 0.21428571428571427,
   'f': 0.09523809178130524}},
 {'rouge-1': {'r': 0.08163265306122448,
   'p': 0.26666666666666666,
   'f': 0.12499999641113292},
  'rouge-2': {'r': 0.03508771929824561,
   'p': 0.13333333333333333,
   'f': 0.05555555225694465},
  'rouge-l': {'r': 0.061224489795918366, 'p': 0.2, 'f': 0.09374999641113295}},
 {'rouge-1': {'r': 0.32653061224489793,
   'p': 0.5714285714285714,
   'f': 0.41558441095631643},
  'rouge-2': {'r': 0.14035087719298245,
   'p': 0.24242424242424243,
   'f': 0.17777777313333343},
  'rouge-l': {'r': 0.30612244897959184,
   'p': 0.5357142857142857,
   'f': 0.3896103849822905}},
 {'rouge-1': {'r': 0.16326530612244897, 'p': 0.32, 'f': 0.2162162117421476},
  'rouge-2': {'r': 0.03508771929824561,
   'p': 0

In [55]:
rouge_scores[0]

{'rouge-1': {'r': 0.061224489795918366,
  'p': 0.21428571428571427,
  'f': 0.09523809178130524},
 'rouge-2': {'r': 0.017543859649122806,
  'p': 0.07142857142857142,
  'f': 0.028169010918468917},
 'rouge-l': {'r': 0.061224489795918366,
  'p': 0.21428571428571427,
  'f': 0.09523809178130524}}

In [58]:
rouge_2_list = [item['rouge-2']  for item in rouge_scores]
    
df_rouge_2 = pd.DataFrame(rouge_2_list)

print(df_rouge_2.head(5))

          r         p         f
0  0.017544  0.071429  0.028169
1  0.035088  0.133333  0.055556
2  0.140351  0.242424  0.177778
3  0.035088  0.071429  0.047059
4  0.070175  0.022346  0.033898


In [59]:
avg_rouge_2 = df_rouge_2.mean()

print(avg_rouge_2)

r    0.198613
p    0.258626
f    0.206965
dtype: float64


In [60]:
np.mean(avg_rouge_2)

0.22140135503422884